In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import os
from crewai import Agent,Task,Crew , LLM
import yaml

In [3]:
llm = LLM(
    model = "ollama/gemma2:2b",
    base_url="http://localhost:11434"
)

In [4]:

files = {
    'agents':'config/agents.yaml',
    'tasks':'config/tasks.yaml'
}

configs={}

for config_type,file_path in files.items():
    with open(file_path,'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config= configs['agents']
tasks_config = configs['tasks']

In [5]:
# Use CrewAI's built-in file reading instead
from crewai_tools import FileReadTool

csv_tool = FileReadTool(
    file_path='config/support_tickets_data.csv',
    start_line= 1,      # no quotes
    line_count= None  
)

/opt/anaconda3/envs/DSenv/lib/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [6]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool],
  llm=llm
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool],
  llm=llm
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=False,
  llm=llm
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True,
  llm=llm
)


In [ ]:
support_report_crew.test(n_iterations=1,eval_llm=llm)

2025-08-12 17:17:46,778 - 8332894400 - __init__.py-__init__:111 - WARNING: Invalid type LLM for attribute 'model_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


╭──────────────────────────────────────────────── Test Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  🧪 Starting Crew Test                                                                                          │
│                                                                                                                 │
│  Crew: crew                                                                                                     │
│  ID: 29347797-8e79-42af-bffb-1147222e4b6f                                                                       │
│  Iterations: 1                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 763802b9-d256-42f6-887d-ec0f3da2cd24                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be  │
│  based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use      │
│  historical data such as resolution_time_minutes and                                                            │
│    satisfaction_rating to inform the suggestions.                                                               │
│  - Customer Feedback: Incorporate insights from customer_comments to                                            │
│    customize the suggestions further.                                                                           │
│                                                                                                                 │
│  The goal is to provide clear, actionable steps that the support team can take to resolve each issue            │
│  efficiently and effectively.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Thought: Thought: I need to access historical data for each ticket to formulate actionable suggestions, then   │
│  incorporate customer feedback for additional customization.                                                    │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"config/support_tickets_data.csv\", \"start_line\": 1, \"line_count\": -1}"                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  I tried reusing the same input, I must stop using this action input. I'll try something else instead.          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Thought: Thought: I need to access historical data for each ticket to formulate actionable suggestions, then   │
│  incorporate customer feedback for additional customization.                                                    │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"config/support_tickets_data.csv\", \"start_line\": 1, \"line_count\": -1}"                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  I tried reusing the same input, I must stop using this action input. I'll try something else instead.          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

In [ ]:
support_report_crew.test(n_iterations=1, eval_llm = llm)

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

In [ ]:
result = support_report_crew.kickoff()

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))